<a href="https://colab.research.google.com/github/m-triassi/ai-projects-472/blob/main/Project%203/Project3_task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import Dependencies
from gensim.downloader import load